In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**This competition is evaluated on a utility score. Each row in the test set represents a trading opportunity for which you will be predicting an action value, 1 to make the trade and 0 to pass on it. Each trade j has an associated weight and resp, which represents a return.**

**We will import the main libraries to evaluate our model. However, we're not use all of them, just ones who brings
us a better results.**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression,RidgeCV, ElasticNet
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [ ]:
df_train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
df_feat = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
df_ex_sam = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_sample_submission.csv')
df_ex = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv')

In [ ]:
def resizeplot(x,y,d):
    plt.figure(figsize=(x,y),dpi=d)

**Let's see our dataset**

In [ ]:
df_train.head()

In [ ]:
df_train.info()

**Let's check if there's some of null data.**

In [ ]:
df_train[['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp','ts_id']].isnull().sum()

**Below is an example of one of our features and how it is distributed.**

In [ ]:
resizeplot(10,6,90)
sns.histplot(x=df_train['weight'],bins=80)

**Let's see how our data is correlated.**

In [ ]:
resp = df_train.iloc[:,2:7]
sns.pairplot(resp)

In [ ]:
resp.describe()

In [ ]:
df_train['action'] =np.where((df_train.resp_1 > 0) & (df_train.resp_2 > 0) & (df_train.resp_3 > 0) & (df_train.resp_4 > 0) & (df_train.resp > 0),1,0)

In [ ]:
X = df_train[['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']]
y = df_train['action']

In [ ]:
scaler = StandardScaler()
scale_X = scaler.fit_transform(X)

In [ ]:
#GRID MODEL/ELASTIC NET

model_elastic = ElasticNet(alpha=0.1,l1_ratio=0.1)
#param_grid_en = {'alpha':[0.1,1,5,10,100,500],'l1_ratio':[.1,.7,.99,1]}
#grid_model_en = GridSearchCV(model_elastic,param_grid = param_grid_en,scoring='neg_mean_squared_error',
#                         cv=5,verbose=1)
model_elastic.fit(scale_X,y)
model_elastic = ElasticNet(alpha=0.1,l1_ratio=0.1)
model_elastic.fit(scale_X,y)

y_pred_en = model_elastic.predict(scale_X)
mae_net = mean_absolute_error(y,y_pred_en)
score_net = r2_score(y,y_pred_en)

print('Score_net: ', score_net)
print('Mae_net', mae_net)

In [ ]:
#XGBOOST

#param_grid_x = {'n_estimators':[100],'verbosity':[1],'eval_metric':['mae'],
             #'max_depth':[6]}
xgb =XGBRegressor(n_estimators=100,verbosity=1,eval_metric='mae', max_depth=6)
#grid_xgb = GridSearchCV(xgb,param_grid_x,n_jobs=-1,cv=3)
xgb.fit(scale_X,y)
pred_xgb = xgb.predict(scale_X)
mae_xgb = mean_absolute_error(y,pred_xgb)
mse_xgb = mean_squared_error(y,pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y,pred_xgb))

score_xgb = r2_score(y,pred_xgb)

print('Score: ',score_xgb)
print('Mean_Absolute_error: ',mae_xgb)
print('Mean_Squared_Error: ',mse_xgb)
print('Root_Mean_Squared_Error: ',rmse_xgb)

In [ ]:
# Neural Nets

mlpr = MLPRegressor(activation= 'logistic',hidden_layer_sizes= (4,4), learning_rate= 'invscaling',
                    max_iter= 1000,solver= 'adam',verbose= True)
#param_grid = {'activation':['identity','logistic','relu'],'solver':['lbfgs', 'sgd', 'adam'],
             #'learning_rate':['constant','invscaling','adaptive'],'max_iter':[1000],
             #'verbose':['True'],'hidden_layer_sizes':[2,2]}

#grid_neural_cv = GridSearchCV(mlpr,param_grid,n_jobs=-1,cv=3)
mlpr.fit(scale_X,y)
grid_pred_neural = mlpr.predict(scale_X)
mae_neural = mean_absolute_error(y,grid_pred_neural)
mse_neural = mean_squared_error(y,grid_pred_neural)
rmse_neural = np.sqrt(mse_neural)
score_neural = r2_score(y,grid_pred_neural)
print('\n')
print('Score_neural: ', score_neural)
print('\n')
print('Mae_neural: ', mae_neural)
print('Mse_neural: ', mse_neural)
print('Rmse_neural: ', rmse_neural)

In [ ]:
# KNN REGRESSOR

knnr = KNeighborsRegressor(algorithm= 'auto', leaf_size= 100, n_neighbors= 15)
#param_knn_grid = {'n_neighbors':[5,15,20],'leaf_size':[100,200],
 #                 'algorithm':['auto']}
#grid_knnr = GridSearchCV(knnr,param_knn_grid,n_jobs=-1,cv=3)
#grid_knnr.fit(X,y)

knnr.fit(scale_X,y)
pred_knnr = knnr.predict(scale_X)

mae_knr = mean_absolute_error(y,pred_knnr)
mse_knr = mean_squared_error(y,pred_knnr)
rmse_knr = np.sqrt(mse_knr)

score_knnr = r2_score(y,pred_knnr)

print('Score: ',score_knnr)
print('\n')
print('Mean_Absolute_error: ',mae_knr)
print('Mean_Squared_Error: ',mse_knr)
print('Root_Mean_Squared_Error: ',rmse_knr)

**Neural Networks shows us the best scores and results.**

In [ ]:
action = pd.DataFrame(grid_pred_neural,columns=['Action'])
submission = pd.concat([df_train['ts_id'],action.astype(int)],axis=1)

In [ ]:
action = np.round(action,1)

In [ ]:
action = action.astype(int)

In [ ]:
action.head(20)

In [ ]:
submission = pd.concat([df_train['ts_id'],action],axis=1)

In [ ]:
submission.tail(20)

In [ ]:
submission.to_csv('submission.csv',index=False)